In [55]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

nlp = spacy.blank('ur')

import numpy as np

In [20]:
faraz = pd.read_excel('Faraz Labels.xlsx')
jibran = pd.read_excel('Jibran Labels.xlsx')

In [21]:
print(faraz.head())
print(jibran.head())

                                                Text Label (h/o)
0                             کہ کے لے لی شام دلے کی           o
1  اب اگر اس نے کچھ جواب دیا تو اس کی گانڈ مار دی...           o
2  اب ان چوتیوں نے وہ جو کنسرٹ ہو رہا تھا وہ بھی ...           h
3  اب ان کی گانڈ میں ہاتھ ڈال کر انتڑیاں نکالے گا...           o
4  اب تو اتنی مار دی تیری بھائی نے کہ اب تو شادی ...           o
                                                Text Label (h/o)
0                             کہ کے لے لی شام دلے کی           o
1  اب اگر اس نے کچھ جواب دیا تو اس کی گانڈ مار دی...           o
2  اب ان چوتیوں نے وہ جو کنسرٹ ہو رہا تھا وہ بھی ...           h
3  اب ان کی گانڈ میں ہاتھ ڈال کر انتڑیاں نکالے گا...           o
4  اب تو اتنی مار دی تیری بھائی نے کہ اب تو شادی ...           o


In [22]:
print(faraz.shape)
print(jibran.shape)
##rename column name Label (h/o) to label
faraz = faraz.rename(columns={'Label (h/o)':'label'})
jibran = jibran.rename(columns={'Label (h/o)':'label'})

(1135, 2)
(1135, 2)


In [15]:
merged_df = pd.merge(faraz, jibran, on='Text', suffixes=('_faraz', '_jibran'))

# Filter rows where the labels don't match
disagreed_df = merged_df[merged_df['label_faraz'] != merged_df['label_jibran']]

# Create a new dataframe with the disagreed data
disagreed_labels_df = pd.DataFrame({
    'Text': disagreed_df['Text'],
    'label_faraz': disagreed_df['label_faraz'],
    'label_jibran': disagreed_df['label_jibran']
})

# Save the new dataframe to a CSV file or any other desired format
disagreed_labels_df.to_csv('disagreed_labels.csv', index=False)

In [18]:
disagreed_labels_df.head()
faraz.head()
jibran.head()


,Text,label
0,کہ کے لے لی شام دلے کی,o
1,اب اگر اس نے کچھ جواب دیا تو اس کی گانڈ مار دی...,o
2,اب ان چوتیوں نے وہ جو کنسرٹ ہو رہا تھا وہ بھی ...,h
3,اب ان کی گانڈ میں ہاتھ ڈال کر انتڑیاں نکالے گا...,o
4,اب تو اتنی مار دی تیری بھائی نے کہ اب تو شادی ...,o


In [23]:
merged_df = pd.merge(faraz, jibran, on='Text', suffixes=('_faraz', '_jibran'))

# Filter rows where the labels don't match
disagreed_df = merged_df[merged_df['label_faraz'] != merged_df['label_jibran']]

# Randomly assign one annotator's label for each disagreed example
random_assignment = np.random.choice(['faraz', 'jibran'], size=len(disagreed_df))
disagreed_df['chosen_annotator'] = random_assignment
disagreed_df['final_label'] = np.where(disagreed_df['chosen_annotator'] == 'faraz', disagreed_df['label_faraz'], disagreed_df['label_jibran'])

# Create a new dataframe with the final assigned labels
final_labels_df = pd.DataFrame({
    'Text': disagreed_df['Text'],
    'final_label': disagreed_df['final_label']
})

# Save the new dataframe to a CSV file or any other desired format
final_labels_df.to_csv('final_assigned_labels.csv', index=False)

C:\Users\jibra\AppData\Local\Temp\ipykernel_28544\3986893872.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  disagreed_df['chosen_annotator'] = random_assignment
C:\Users\jibra\AppData\Local\Temp\ipykernel_28544\3986893872.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  disagreed_df['final_label'] = np.where(disagreed_df['chosen_annotator'] == 'faraz', disagreed_df['label_faraz'], disagreed_df['label_jibran'])


In [29]:
faraz = faraz[~faraz['Text'].isin(final_labels_df['Text'])]
jibran = jibran[~jibran['Text'].isin(final_labels_df['Text'])]
print(faraz.shape, jibran.shape)
#merge faraz, jibran, final_labels_df removing duplicates

(991, 2) (991, 2)


In [30]:
merged_df = pd.concat([jibran, final_labels_df]).drop_duplicates()


In [40]:

merged_df.shape
#write merged_df to csv
merged_df.to_csv('combined_dataset.csv', index=False)


In [42]:
merged_df.head()

,Text,label
0,کہ کے لے لی شام دلے کی,o
1,اب اگر اس نے کچھ جواب دیا تو اس کی گانڈ مار دی...,o
2,اب ان چوتیوں نے وہ جو کنسرٹ ہو رہا تھا وہ بھی ...,h
3,اب ان کی گانڈ میں ہاتھ ڈال کر انتڑیاں نکالے گا...,o
4,اب تو اتنی مار دی تیری بھائی نے کہ اب تو شادی ...,o


In [37]:
mob_data = pd.read_csv('Hs Urdu - HS Urdu.csv')
mob_data.head()

,Text,Label
0,دس ڈالر تو حبشی کا چُوپہ لگا کر گوں رایہ حرامی...,o
1,تیری کڑی دی پھدی چے حبشی دا لن گانڈو باپ کی او...,o
2,تم تو کسی ہندو مندر کے پجاری کی فصل ہو جو کُم ...,h
3,بيشرم بيغيرت ہندو مودی کے حبشی بچے تيرے جيسے ک...,h
4,انڈين مودی کے عمار حبشی پِلے کيا يہ رنگ بازی ت...,o


In [48]:
#renamed merge_data label to Label
merged_df = merged_df.rename(columns={'label':'Label'})

In [50]:
#combine merged_data and mob_data
combined_df = pd.concat([merged_df, mob_data])

In [52]:
# combined_df.head()
combined_df.shape

(9005, 2)

In [54]:
# write combined_df to final_combined.csv
combined_df.to_csv('final_combined_mobandus.csv', index=False)

In [56]:
combined_df.to_excel('final_combined_mobandus.xlsx', index=False)